In [1]:
from common_functions import ret_metabase, google_sheets, upload_dataframe_to_snowflake, snowflake_query, upload_dataframe_to_pg, dwh_query, get_secret
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
# import psycopg2
import numpy as np
import gspread
import sqlalchemy
import pytz
import psycopg2

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
def initialize_env():
    """
    Initialize environment variables and credentials for Snowflake, Slack, Metabase, and Google Sheets.
    Reads secrets and sets them as environment variables for use in other functions.
    """
    snowflake_sg_secret = json.loads(get_secret("Snowflake-sagemaker"))
    slack_secret = json.loads(get_secret("prod/slack/reports"))
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    dwh_writer_secret = json.loads(get_secret("prod/db/datawarehouse/sagemaker"))

    os.environ["SNOWFLAKE_USERNAME"] = snowflake_sg_secret["username"]
    os.environ["SNOWFLAKE_PASSWORD"] = snowflake_sg_secret["password"]
    os.environ["SNOWFLAKE_ACCOUNT"] = snowflake_sg_secret["account"]
    os.environ["SNOWFLAKE_DATABASE"] = snowflake_sg_secret["database"]

    os.environ["SLACK_TOKEN"] = slack_secret["token"]

    os.environ["FINTECH_EMONEY_EMAIL"] = fintech_service_account["email_name"]
    os.environ["FINTECH_EMONEY_PASSWORD"] = fintech_service_account["email_password"]

    metabase_secret = json.loads(get_secret("prod/metabase/maxab_config"))
    os.environ["EGYPT_METABASE_USERNAME"] = metabase_secret["metabase_user"]
    os.environ["EGYPT_METABASE_PASSWORD"] = metabase_secret["metabase_password"]

    os.environ["DWH_WRITER_HOST_NEW"] = dwh_writer_secret["host"]
    os.environ["DWH_WRITER_NAME_NEW"] = dwh_writer_secret["dbname"]
    os.environ["DWH_WRITER_USER_NAME_NEW"] = dwh_writer_secret["username"]
    os.environ["DWH_WRITER_PASSWORD_NEW"] = dwh_writer_secret["password"] 

    json_path_sheets = str(Path.home()) + "/service_account_key_sheets.json"
    sheets_key = get_secret("prod/maxab-sheets")
    f = open(json_path_sheets, "w")
    f.write(sheets_key)
    f.close()
    os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"] = json_path_sheets


In [3]:
initialize_env()

host = os.environ["DWH_WRITER_HOST_NEW"]
database = os.environ["DWH_WRITER_NAME_NEW"]
user = os.environ["DWH_WRITER_USER_NAME_NEW"]
password = os.environ["DWH_WRITER_PASSWORD_NEW"]

conn = psycopg2.connect(host=host, database=database, user=user, password=password)
print("Successfully connected to DB")

Successfully connected to DB


In [4]:
    engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{database}")
    print(bool(engine))

True


In [5]:
agent_group = google_sheets("Agents - Retailers", "Agent-to-group", "get")
col_list = agent_group.columns.tolist()
col_list

/home/ec2-user/service_account_key.json


['Benchmark',
 'Somaya Alrab',
 'Esraa Mohamed',
 'Ebthal Saber',
 'Raneen Ali',
 'Abdelrahman Merghany',
 'Israa Abdelhamid',
 'Nada Saber',
 'Marina Riyad']

In [6]:
def get_available_agents(attendance_df, current_hour):
    """
    Get a list of available task-based agents for the current hour based on attendance DataFrame.
    Args:
        attendance_df (pd.DataFrame): DataFrame with agent attendance info
        current_hour (int): Current hour (24-hour format)
    Returns:
        list: List of available agent IDs
    """
    attendance_copy = attendance_df.copy()
    
    attendance_copy['start_time'] = attendance_copy['start_time'].astype(int)
    attendance_copy['end_time'] = attendance_copy['end_time'].astype(int)
    
    attendance_copy['assignment_start_time'] = attendance_copy['start_time'] - 1
    attendance_copy['assignment_end_time'] = attendance_copy['end_time'] - 1
    
    attendance_copy['assign_data'] = np.where(
        (current_hour >= attendance_copy['assignment_start_time']) & 
        (current_hour <= attendance_copy['assignment_end_time']),
        'yes', 'no')
    
    task_based_agents = attendance_copy.loc[
        (attendance_copy['project'] == 'task_based') & 
        (attendance_copy['assign_data'] == 'yes')]
    
    task_based_list = task_based_agents['agent_id'].values.tolist()
    print(f"Number of available agents: {len(task_based_list)}")
    return task_based_list


def assign_data_equal_projects(df, list):
    df = df.sample(frac=1)  # Shuffle the data
    project_types = df['project_name'].unique()
    
    assigned_data = pd.DataFrame()
    
    for project in project_types:
        project_df = df[df['project_name'] == project]
        project_df = project_df.reset_index(drop=True)
        rows_per_agent = len(project_df) // len(list)
        remainder = len(project_df) % len(list)
        
        # Distribute rows equally
        for i, agent in enumerate(list):
            start_idx = i * rows_per_agent
            end_idx = start_idx + rows_per_agent
            agent_data = project_df.iloc[start_idx:end_idx].copy()
            agent_data['agent_assigned'] = agent
            
            # Handle remainder
            if i < remainder:
                extra_row = project_df.iloc[end_idx:end_idx+1].copy()
                extra_row['agent_assigned'] = agent
                agent_data = pd.concat([agent_data, extra_row])
            
            assigned_data = pd.concat([assigned_data, agent_data])
    
    assigned_data = assigned_data.reset_index(drop=True)
    
    return assigned_data

In [7]:
now = datetime.now() + timedelta(hours=3)
hour = int(str(now.time())[0:2])
attendance = ret_metabase("EGYPT", 13502)
print(f"Total agents: {len(attendance)}")
task_based_list = get_available_agents(attendance, hour)

/home/ec2-user/service_account_key.json
Total agents: 5
Number of available agents: 5


In [17]:
Data = google_sheets("POS Campaign | July & August 2025", "Main", "Get")

task_based_list = get_available_agents(attendance, hour)
# Convert Date column
Data["Date"] = pd.to_datetime(Data["Date"], format="%m/%d/%Y")
# Get today's date (normalized to midnight)
today = pd.Timestamp.today().normalize()
# Filter for today's rows
Data_filtered = Data[Data["Date"] == today]

sheet_Data = Data_filtered.copy()
sheet_Data.rename(columns={'mobile': 'main_system_id','name': 'retailer_name'},inplace=True)

sheet_Data["main_system_id"] = sheet_Data["main_system_id"].astype('Int64').astype(str)
# Remove leading '2' if present
sheet_Data["main_system_id"] = sheet_Data["main_system_id"].str.replace(r"^2", "", regex=True)

# Drop duplicate main_system_id values (keeping first occurrence)
sheet_Data = sheet_Data.drop_duplicates(subset="main_system_id", keep="first")

sheet_Data["description"] = (
    "POS Acquisition Marketing - location: "
    + sheet_Data["location"].astype(str)
    + " - campaign:"
    + sheet_Data["campaign"].astype(str)
)

sheet_Data.drop(columns=["location", "campaign", "Date"], inplace=True)
sheet_Data['Added_at'] = now
sheet_Data['project_name'] = "POS Leads"
name_list = ['Somaya Alrab','Esraa Mohamed','Marina Riyad','Abdelrahman Merghany','Israa Abdelhamid','Ebthal Saber']
Pos_leads = assign_data_equal_projects(sheet_Data , name_list)
data_for_sql = assign_data_equal_projects(sheet_Data, task_based_list)
data_for_sql.rename(columns={'main_system_id': 'offer'},inplace=True)
data_for_sql['main_system_id'] = 11111
# Check if description column exists before dropping it
if "description" in data_for_sql.columns:
    data_for_sql.drop(columns=["description"], inplace=True)
data_for_sql["description"] = "POS Acquisition Marketing - campaign: " + Data["campaign"]
data_for_sql = data_for_sql[['main_system_id', 'description','offer','agent_assigned']]
data_for_sql['dispatched_at'] = now
data_for_sql['agent_assigned'] = data_for_sql['agent_assigned'].astype(int) 
google_sheets("Agents - Retailers", "Task_based", "append", df=Pos_leads)

for i in col_list:
    if i == "Benchmark":
        continue 
    filtered_df = Pos_leads[Pos_leads["agent_assigned"] == i]
    filtered_df.drop(columns=["agent_assigned"], inplace=True)
    print(f"Assigned to {i},{len(filtered_df)} Task-based Tasks")
    google_sheets(i, "Task_based", "append", df=filtered_df)

with engine.connect() as conn:
    data_for_sql.to_sql('task_based_am_projects', schema='fintech', con=engine, if_exists='append', chunksize=1000, method='multi', index=False)


/home/ec2-user/service_account_key.json
Number of available agents: 5
/home/ec2-user/service_account_key.json


/tmp/ipykernel_14981/3591926963.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(columns=["agent_assigned"], inplace=True)


Assigned to Somaya Alrab,64 Task-based Tasks
/home/ec2-user/service_account_key.json


/tmp/ipykernel_14981/3591926963.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(columns=["agent_assigned"], inplace=True)


Assigned to Esraa Mohamed,64 Task-based Tasks
/home/ec2-user/service_account_key.json


...

/tmp/ipykernel_14981/3591926963.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(columns=["agent_assigned"], inplace=True)


Assigned to Nada Saber,0 Task-based Tasks
/home/ec2-user/service_account_key.json


/home/ec2-user/SageMaker/git_files/Seif/Seif's_stuff/common_functions.py:285: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_data = pd.concat([existing_data, df], ignore_index=True)
/tmp/ipykernel_14981/3591926963.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(columns=["agent_assigned"], inplace=True)


Assigned to Marina Riyad,64 Task-based Tasks
/home/ec2-user/service_account_key.json


/home/ec2-user/SageMaker/git_files/Seif/Seif's_stuff/common_functions.py:285: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_data = pd.concat([existing_data, df], ignore_index=True)


In [14]:
Data_filtered

,Date,name,mobile,location,campaign
2509,2025-10-12,Alaa Sayed,201207394811,Alexandria,Alex
2510,2025-10-12,ابو اسيل ولانا,201223002444,Mansoura,Alex
2511,2025-10-12,مدحت عزمي ثابت,201141350803,اسيوط مركز صدفا اولاد الياس,Alex
2512,2025-10-12,صلاح محمد دياب,201066957120,Ismailia,Alex
2513,2025-10-12,احمد سوهاج,201004092239,جرجا,Alex
...,...,...,...,...,...
2889,2025-10-12,دهب عطية ضاحي,201126254690,القاهرة,Cairo & Giza
2890,2025-10-12,Mohamad Hosny,201117700737,Cairo,Cairo & Giza
2891,2025-10-12,Ahmed Salem,201116076265,Cairo,Cairo & Giza
2892,2025-10-12,Shnoda Abdallah,201017139132,Cairo,Cairo & Giza


In [15]:
sheet_data = Data_filtered.sort_values(by='mobile', ascending=False)
sheet_data

,Date,name,mobile,location,campaign
2598,2025-10-12,Hussien Madany Hassan Mahmuod,20974620020,Aswan,Alex
2824,2025-10-12,ابو فارس الامير ابوحشيش,2032904004,Sohag,Cairo & Giza
2857,2025-10-12,محمدخلیل,20242107023,Qalyub,Cairo & Giza
2772,2025-10-12,محمدجبريل ابوالفضل,201288694794,1288694794,Cairo & Giza
2720,2025-10-12,ابو يوسف الجز,201287997669,Benha,Cairo & Giza
...,...,...,...,...,...
2537,2025-10-12,Ahmed Elsharkawy,201000377444,Tanta,Alex
2752,2025-10-12,الطيب ابوالطيب,201000350107,Zagazig,Cairo & Giza
2655,2025-10-12,محمدعاشور عيسى,201000271540,Abu el-Matamir el-Bahari,Alex
2757,2025-10-12,احمد عمر مكرم,"#VALUE! (In FIND evaluation, cannot find '20' ...",اسيوط منقباد العدر,Cairo & Giza
